In [18]:
# lat, lon = 45.087195, 7.653585
import geopandas as gpd
from setuptools.compat.py311 import shutil_rmtree
from shapely.geometry import Point
import requests
import zipfile
import os
import osmnx as ox

address = "Turin, Italy"

# Geocode the address to get the coordinates
location = ox.geocode(address)
lat, lon = location[1], location[0]

# Path of the shapefile
shapefile_path = "tiling_schema/GHSL2_0_MWD_L1_tile_schema_land.shp"
extracted_dir = "extracted_files"

# Remove the directory if it already exists
if os.path.exists(extracted_dir):
    shutil_rmtree(extracted_dir)

# Check the CRS of the shapefile
tiles_gdf = gpd.read_file(shapefile_path)
print(f"Shapefile CRS: {tiles_gdf.crs}")

point = gpd.GeoDataFrame(geometry=[Point(lon, lat)], crs="EPSG:4326")
point = point.to_crs(tiles_gdf.crs)

current_tile = tiles_gdf[tiles_gdf.geometry.contains(point.geometry.iloc[0])]

# Extract the tile id
if not current_tile.empty:
    tile_id = current_tile.iloc[0]['tile_id']
    print(f"Tile ID: {tile_id}")

    # URL for downloading the tile
    url = f"https://jeodpp.jrc.ec.europa.eu/ftp/jrc-opendata/GHSL/GHS_POP_GLOBE_R2023A/GHS_POP_E2020_GLOBE_R2023A_54009_1000/V1-0/tiles/GHS_POP_E2020_GLOBE_R2023A_54009_1000_V1_0_{tile_id}.zip"

    response = requests.get(url)

    # Save the zip file
    if response.status_code == 200:
        zip_path = 'tile_download.zip'
        with open(zip_path, 'wb') as file:
            file.write(response.content)

        # Estract only the .tif file
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            for file in zip_ref.namelist():
                if file.endswith('.tif'):
                    zip_ref.extract(file, extracted_dir)
                    tif_path = os.path.join(extracted_dir, file)
                    new_tif_path = os.path.join(extracted_dir, 'current_tile.tif')
                    os.rename(tif_path, new_tif_path)
        os.remove(zip_path)

    else:
        print("Error while downloading the tile")

Shapefile CRS: None


ValueError: Must pass either crs or epsg.